In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
path ="data/"
app = pd.read_table(path+'app_launch_log.txt',names=['user_id','day'],encoding='utf-8',sep='\t',)
user_act = pd.read_table(path+'user_activity_log.txt',names=['user_id','day','page','video_id','author_id','action_type'],encoding='utf-8',sep='\t')
user_reg = pd.read_table(path+'user_register_log.txt',names=['user_id','register_day','register_type','device_type'],encoding='utf-8',sep='\t')
video = pd.read_table(path+'video_create_log.txt',names=['user_id','day'],encoding='utf-8',sep='\t')

In [3]:
app.to_csv('data/app_lauch_log.csv',index=False)
user_act.to_csv('data/user_activity_log.csv',index=False)
user_reg.to_csv('data/user_register_log.csv',index=False)
video.to_csv('data/video_create_log.csv',index=False)

In [4]:
app = pd.read_csv('data/app_lauch_log.csv')
user_act = pd.read_csv('data/user_activity_log.csv')
user_reg = pd.read_csv('data/user_register_log.csv')
video = pd.read_csv('data/video_create_log.csv')

In [4]:
# total_info = pd.DataFrame({'user_id':np.unique(pd.concat([app.user_id,user_act.user_id,user_reg.user_id,video.user_id]))})
# a = app.groupby('user_id').count()['day']
# a = pd.DataFrame(a.reset_index())
# a.rename(columns={'day':'total_launch'}, inplace=True)
# a
# b = video.groupby('user_id').count()['day']
# b = pd.DataFrame(b.reset_index())
# b.rename(columns={'day':'total_create'},inplace=True)
# c = user_act.groupby('user_id').count()['day']
# c = pd.DataFrame(c.reset_index())
# c.rename(columns={'day':'total_act'}, inplace=True)
# total_info = pd.merge(total_info,a,on='user_id',how='left')
# total_info = pd.merge(total_info,b,on='user_id',how='left')
# total_info = pd.merge(total_info,c,on='user_id',how='left')
# total_info.head(50)
# total_info.to_csv('data/total_info.csv')
app.head()

,user_id,day
0,1287915,3
1,1287915,4
2,1287915,8
3,1287915,14
4,1287915,15


In [5]:
vid = pd.DataFrame(user_act.video_id)
vid['hot'] = 1
v = vid.groupby('video_id').count()
v.reset_index(inplace=True)
v_norm = (v.hot - v.hot.min()) / (v.hot.max() - v.hot.min())
v['video_norm_hot']= v_norm
v.rename(columns={'hot':'vid_hotness'}, inplace=True)
user_act = pd.merge(user_act,v,on='video_id',how='left')

In [6]:
user_act.head()

,user_id,day,page,video_id,author_id,action_type,vid_hotness,video_norm_hot
0,1185527,13,0,2685033,1150188,0,22,0.005730
1,282033,14,1,331791,400971,0,1,0.000000
2,209866,15,0,4081958,541368,0,2,0.000273
3,802521,13,0,4081958,541368,0,2,0.000273
4,342650,13,0,3705772,1301397,0,1,0.000000


In [7]:
aut = pd.DataFrame(user_act.author_id)
aut['hot'] = 1
a = aut.groupby('author_id').count()
a.reset_index(inplace=True)
a_norm = (a.hot - a.hot.min()) / (a.hot.max() - a.hot.min())
a['author_norm_hot'] = a_norm
a.rename(columns={'hot':'auth_hotness'}, inplace=True)
user_act = pd.merge(user_act,a,on='author_id',how='left')
user_act.to_csv('data/user_activity_log.csv', index=False)
user_act.head()

,user_id,day,page,video_id,author_id,action_type,vid_hotness,video_norm_hot,auth_hotness,author_norm_hot
0,1185527,13,0,2685033,1150188,0,22,0.005730,278,0.010029
1,282033,14,1,331791,400971,0,1,0.000000,153,0.005503
2,209866,15,0,4081958,541368,0,2,0.000273,2,0.000036
3,802521,13,0,4081958,541368,0,2,0.000273,2,0.000036
4,342650,13,0,3705772,1301397,0,1,0.000000,1,0.000000


In [8]:
a.rename(columns={'auth_hotness':'hotness_as_creater','author_id':'user_id','author_norm_hot':'norm_hotness_as_creater'}, inplace=True)
a.head()

,user_id,hotness_as_creater,norm_hotness_as_creater
0,1,11,0.000362
1,2,1,0.000000
2,3,1,0.000000
3,4,1,0.000000
4,5,1,0.000000


In [9]:
video = pd.merge(video, a, on='user_id',how='left')
video.head()
video.to_csv('data/video_create_log.csv',index=False)

In [10]:
A_train = 'data/1_data_train_'
B_train = 'data/2_data_train_'
C_train = 'data/3_data_train_'
A_test = 'data/1_data_test_'
B_test = 'data/2_data_test_'

In [11]:
def cut_data_as_time(new_data_path, begin_day, end_day):
    temp_register = user_reg[(user_reg['register_day'] >= 1) & (user_reg['register_day'] <= end_day)]
    temp_create = video[(video['day'] >= begin_day) & (video['day'] <= end_day)]
    temp_launch = app[(app['day'] >= begin_day) & (app['day'] <= end_day)]
    temp_activity = user_act[(user_act['day'] >= begin_day) & (user_act['day'] <= end_day)]
    
    temp_register.to_csv(new_data_path + "register.csv", index = False)
    temp_activity.to_csv(new_data_path + "activity.csv", index = False)
    temp_launch.to_csv(new_data_path + "launch.csv", index = False)
    temp_create.to_csv(new_data_path + "create.csv", index = False)
    print('根据起始和结束时间将数据集取出完成,存入:',new_data_path)

In [12]:
def generate_dataset():
    print('开始划分数据集....')
    begin_day = 1
    end_day = 16
    cut_data_as_time(A_train,begin_day, end_day)
    begin_day = 17
    end_day = 23
    cut_data_as_time(A_test,begin_day, end_day)
    print('第一数据集划分完成')
    begin_day = 8
    end_day = 23
    cut_data_as_time(B_train,begin_day, end_day)
    begin_day = 24
    end_day = 30
    cut_data_as_time(B_test,begin_day, end_day)
    print('第二数据集划分完成')
    begin_day = 15
    end_day = 30
    cut_data_as_time(C_train,begin_day, end_day)
    print('第三数据集划分完成')
    print('全部数据集划分完成.....')

In [13]:
generate_dataset()

开始划分数据集....
根据起始和结束时间将数据集取出完成,存入: data/1_data_train_
根据起始和结束时间将数据集取出完成,存入: data/1_data_test_
第一数据集划分完成
根据起始和结束时间将数据集取出完成,存入: data/2_data_train_
根据起始和结束时间将数据集取出完成,存入: data/2_data_test_
第二数据集划分完成
根据起始和结束时间将数据集取出完成,存入: data/3_data_train_
第三数据集划分完成
全部数据集划分完成.....


In [14]:
video.head()

,user_id,day,hotness_as_creater,norm_hotness_as_creater
0,123785,6,56.0,0.001991
1,123785,5,56.0,0.001991
2,123785,8,56.0,0.001991
3,123785,5,56.0,0.001991
4,123785,5,56.0,0.001991


In [15]:
user_act.tail()

,user_id,day,page,video_id,author_id,action_type,vid_hotness,video_norm_hot,auth_hotness,author_norm_hot
21072398,592563,30,2,1773916,209552,0,2,0.000273,4,0.000109
21072399,592563,30,2,1773916,209552,0,2,0.000273,4,0.000109
21072400,214058,30,1,450181,321228,0,1,0.000000,1,0.000000
21072401,19193,30,1,3180222,437030,0,2,0.000273,83,0.002969
21072402,19193,30,1,3180222,437030,1,2,0.000273,83,0.002969


In [16]:
user_act.author_id.max()

1383751

In [14]:
video.max()

user_id    1367473
day             30
dtype: int64

In [17]:
a = pd.read_csv('data/1_data_train_register.csv')
b = pd.read_csv('data/2_data_train_register.csv')
c = pd.read_csv('data/3_data_train_register.csv')

In [22]:
a.plot()

In [49]:
video.head()

,user_id,day,index,hotness_as_creater
0,720497,1,696685.0,0.028227
1,720497,1,696685.0,0.028227
2,720497,1,696685.0,0.028227
3,1075211,6,1039739.0,2.575683
4,1075211,12,1039739.0,2.575683


In [42]:
a.head()

,index,user_id,hotness_as_creater
0,0,1,0.000000
1,1,2,0.000000
2,2,3,0.000000
3,3,4,0.028227
4,4,5,0.000000


In [23]:
us = user_reg.user_id.drop_duplicates()

In [25]:
us.to_csv("result/full.csv",index=False)

In [29]:
len(us)

51480